In [1]:
#***          Start ib_insync (run once)       *****
#_______________________________________________

from ib_insync import *
util.startLoop()
ib = IB().connect('127.0.0.1', 4001, clientId=12) # kavi IBG live

In [173]:
import pandas as pd
import numpy as np
import datetime
from os import listdir

#... set limits and penalties
m_maxp = 1/100    # max margin allowed per scrip. e.g 1/100 = 1% of net liquidation value
min_rom = 3       # 8 would be 800% return

#... read the account info
ac = ib.accountSummary()
df_a = util.df(ac)

#... set max margin per position
net_liq = float(df_a[df_a.tag == 'NetLiquidation'].iloc[0].value)
max_p = net_liq*m_maxp

#... read the positions
ps = ib.portfolio()
df_p = util.df(ps)

df_p['ibSymbol'] = [s.symbol for s in df_p.contract.values]

#... get the lots and margins

# from 5paisa
paisaurl = "https://www.5paisa.com/5pit/spma.asp"
df_paisa = pd.read_html(paisaurl, header=0)[1].drop_duplicates(subset='Symbol')

# Rename Symbol and Margin fields
df_paisa = df_paisa.rename(columns={'Symbol': 'nseSymbol', 'TotMgn%': 'marginpct'})

# Convert columns to numeric and make margin to pct
df_paisa = df_paisa.apply(pd.to_numeric, errors='ignore')
df_paisa.marginpct = df_paisa.marginpct.div(100)

# Truncate to 9 characters for ibSymbol
df_paisa['ibSymbol'] = df_paisa.nseSymbol.str.slice(0,9)

# nseSymbol to ibSymbol dictionary for conversion
ntoi = {'M&M': 'MM', 'M&MFIN': 'MM', 'L&TFH': 'LTFH', 'NIFTY': 'NIFTY50'}

# remap ibSymbol, based on the dictionary
df_paisa.ibSymbol = df_paisa.ibSymbol.replace(ntoi)

df_slm = pd.merge(df_p, df_paisa[['ibSymbol', 'Mlot', 'TotMgnPerShr']])

#... make the blacklist

df1 = df_slm.groupby('ibSymbol').sum()
df1['used_margin'] = -df1.position * df1.TotMgnPerShr
df1['max_margin'] = max_p
df1['avail_margin'] = df1.position * df1.TotMgnPerShr + df1.max_margin

df1['max_units'] = (df1.avail_margin/(df1.Mlot*df1.TotMgnPerShr)).apply(np.floor)


fspath = './zdata/nse'
oldfspath = './zdata/nse/2018-12-24'

fs = listdir(fspath)
oldfs = listdir(oldfspath)

# Take only pickle files. Remove directories
fs = [f for f in fs if f[-3:] == 'pkl']

# Remove fs from oldfs
oldfs = [f for f in oldfs if f not in fs]

dfsold = pd.concat([pd.read_pickle(oldfspath +'/' + f) for f in oldfs], sort=False).reset_index(drop=True)

penalty = 1.8

# Expected price is rounded up to 0.05 after putting the penalty
dfsold['expPrice'] = round(dfsold[['bid', 'ask', 'close']].max(axis=1)*penalty * 2, 1)/2

dfsold['dte'] = (pd.to_datetime(dfsold.expiry) - datetime.datetime.now()).dt.days

# make negative dtes to 1 to accommodate last day option expiries
dfsold.loc[dfsold.dte <= 0, 'dte'] = 1

dfsold['rom'] = (dfsold.expPrice*dfsold.lot)/dfsold.margin*252/dfsold.dte

dfnew = pd.concat([pd.read_pickle(fspath + '/' + f) for f in fs], sort=False).reset_index(drop=True)

# remove old fields not in new
dfsold2 = dfsold1.drop(['undHi', 'undLo', 'undMean'], axis=1)

# make the mega dataframe
df1 = pd.concat([dfnew, dfsold2], axis=0, sort=False).sort_values(by=['rom'], ascending=False).reset_index(drop=True)

# filter out only this month's expiry
df2 = df1[df1.dte <= 1].sort_values(by=['rom'], ascending=False).reset_index(drop=True)

# Get the top qantile targets
df3 = df2[df2.rom > df2.rom.quantile(q=0.90)]

#... put strikes which are smallest % from undLo 

dfputs = df3[df3.right == 'P'].reset_index(drop=True)
dfputs['put_threshold'] = (dfputs.lo52 - dfputs.strike)/dfputs.strike
dfputs = dfputs.sort_values('put_threshold', ascending=True)

dfcalls = df3[df3.right == 'C'].reset_index(drop=True)
dfcalls['call_threshold'] = (dfcalls.strike - dfcalls.hi52)/dfcalls.strike
dfcalls = dfcalls.sort_values('call_threshold', ascending=True)

In [182]:
df = dfcalls[(dfcalls.call_threshold > 0.1) & (dfcalls.undPrice < dfcalls.meanPrice)]

In [183]:
contracts = [c.contract for c in df.ticker]
orders = [LimitOrder(action='SELL', totalQuantity=lot, lmtPrice=expPrice) for lot, expPrice in zip(df.lot, df.expPrice)]
len(contracts)

16

In [101]:
dfputs1.loc[dfputs1.ibSymbol == 'NHPC', ['ibSymbol', 'expiry', 'strike', 'right','undPrice', 'expPrice', 'dte','rom', 'hi52','lo52','meanPrice','stDev','threshold']]

,ibSymbol,expiry,strike,right,undPrice,expPrice,dte,rom,hi52,lo52,meanPrice,stDev,threshold
81,NHPC,20181227,21.0,P,25.95,0.1,1,5.54151,34.475,22.125,26.333958,2.25246,0.053571
82,NHPC,20181227,20.0,P,25.95,0.1,1,5.54151,34.475,22.125,26.333958,2.25246,0.106250
84,NHPC,20181227,19.0,P,25.95,0.1,1,5.54151,34.475,22.125,26.333958,2.25246,0.164474
80,NHPC,20181227,18.0,P,25.95,0.1,1,5.54151,34.475,22.125,26.333958,2.25246,0.229167
88,NHPC,20181227,17.0,P,25.95,0.1,1,5.54151,34.475,22.125,26.333958,2.25246,0.301471
83,NHPC,20181227,16.0,P,25.95,0.1,1,5.54151,34.475,22.125,26.333958,2.25246,0.382812
85,NHPC,20181227,15.0,P,25.95,0.1,1,5.54151,34.475,22.125,26.333958,2.25246,0.475000
87,NHPC,20181227,14.0,P,25.95,0.1,1,5.54151,34.475,22.125,26.333958,2.25246,0.580357
86,NHPC,20181227,13.0,P,25.95,0.1,1,5.54151,34.475,22.125,26.333958,2.25246,0.701923


In [163]:
ib.accountSummary()

[AccountValue(account='U9329809', tag='AccountType', value='INDIVIDUAL', currency='', modelCode=''),
 AccountValue(account='U9329809', tag='Cushion', value='0.443159', currency='', modelCode=''),
 AccountValue(account='U9329809', tag='LookAheadNextChange', value='1545817500', currency='', modelCode=''),
 AccountValue(account='U9329809', tag='AccruedCash', value='0.00', currency='INR', modelCode=''),
 AccountValue(account='U9329809', tag='AvailableFunds', value='7163180.33', currency='INR', modelCode=''),
 AccountValue(account='U9329809', tag='BuyingPower', value='15264054.88', currency='INR', modelCode=''),
 AccountValue(account='U9329809', tag='EquityWithLoanValue', value='21692079.45', currency='INR', modelCode=''),
 AccountValue(account='U9329809', tag='ExcessLiquidity', value='10444194.48', currency='INR', modelCode=''),
 AccountValue(account='U9329809', tag='FullAvailableFunds', value='7163180.33', currency='INR', modelCode=''),
 AccountValue(account='U9329809', tag='FullExcessLiq

In [185]:
ib.fills()

[Fill(contract=Option(conId=335451257, symbol='MUTHOOTFI', lastTradeDateOrContractMonth='20181227', strike=350.0, right='P', multiplier='1', exchange='NSE', currency='INR', localSymbol='MUTHOOTFIN18DEC350PE', tradingClass='MUTHOOTFIN'), execution=Execution(execId='00016710.5c22ed43.01.01', time=datetime.datetime(2018, 12, 26, 5, 20, 59, tzinfo=datetime.timezone.utc), acctNumber='U9329809', exchange='NSE', side='BOT', shares=1500.0, price=0.05, permId=733814691, clientId=12, orderId=168, cumQty=1500.0, avgPrice=0.05), commissionReport=CommissionReport(execId='00016710.5c22ed43.01.01', commission=23.64575, currency='INR', realizedPNL=480.68825), time=datetime.datetime(2018, 12, 26, 5, 20, 59, 992870, tzinfo=datetime.timezone.utc)),
 Fill(contract=Option(conId=335364471, symbol='CANBK', lastTradeDateOrContractMonth='20181227', strike=190.0, right='P', multiplier='1', exchange='NSE', currency='INR', localSymbol='CANBK18DEC190PE', tradingClass='CANBK'), execution=Execution(execId='00016710.